<a href="https://colab.research.google.com/github/smdbg/colab/blob/main/OWUI_tool_LANCEDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import json
from pydantic import BaseModel, Field

class Tools:
    class Valves(BaseModel):
        api_url: str = Field(
            default="http://host.docker.internal:8000/search",
            description="Пълният URL на LanceDB API сървъра."
        )
        default_limit: int = Field(
            default=5,
            description="Брой резултати по подразбиране."
        )

    def __init__(self):
        print("Initializing LanceDB API Tool (Raw Mode)")
        self.valves = self.Valves()
        self.citation = True

    def search_knowledge_base(self, query: str, limit: int = None) -> str:
        """
        Search the LanceDB knowledge base via API and return raw JSON results.
        :param query: The search query string.
        :param limit: (Optional) Number of results to return.
        :return: A JSON formatted string of the results.
        """
        print(f"Sending query to LanceDB API: {query}")

        target_limit = limit if limit else self.valves.default_limit

        payload = {
            "text": query,
            "limit": target_limit
        }

        try:
            response = requests.post(self.valves.api_url, json=payload, timeout=10)

            # Хваща всички грешки (4xx, 5xx) автоматично
            response.raise_for_status()

            results = response.json()

            if not results:
                return json.dumps({"message": "No documents found."})

            # Връщаме суровия JSON, LLM се оправя с дистанциите
            return f"Query executed successfully. Results in JSON format:\n\n{json.dumps(results, indent=2, ensure_ascii=False)}"

        except Exception as e:
            # Връщаме грешката като JSON, за да знае LLM какво се е случило
            return json.dumps({"error": str(e)})